In [1]:
# Import libraries
import json
import os
import re
import requests
import re
import time
import pandas as pd
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

In [2]:
# Input parameters to change
city = 'Atlanta'
page_num = 5
state = 'GA'

In [46]:
def create_url_hashmap(city: str, pag_num: int, state: str) -> dict:
    """
    Creates a hashmap of the urls
    """
    
    pages = {}

    for i in range(pag_num+1):
        key=str('url' + str(i))
        pages[key] = "https://www.zillow.com/homes/{}-{}/{}_p/".format(city, state, i+1)
        pages[key]=str(pages[key])

    return pages

In [47]:
url_mapping = create_url_hashmap(city, page_num, state)

In [48]:
url_mapping

{'url0': 'https://www.zillow.com/homes/Atlanta-GA/1_p/',
 'url1': 'https://www.zillow.com/homes/Atlanta-GA/2_p/',
 'url2': 'https://www.zillow.com/homes/Atlanta-GA/3_p/',
 'url3': 'https://www.zillow.com/homes/Atlanta-GA/4_p/',
 'url4': 'https://www.zillow.com/homes/Atlanta-GA/5_p/',
 'url5': 'https://www.zillow.com/homes/Atlanta-GA/6_p/'}

In [49]:
# Dictionary of HTTP headers
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [61]:
# Creatng Request Variables and iterating web pages for JSON data. 
with requests.Session() as s:
    zillow_data = []
    for i in range(0, page_num + 1):
        time.sleep(5)
        r0 = s.get(url0, headers=req_headers)
        data = json.loads(re.search(r'!--(\{"queryState".*?)-->', r0.text).group(1))
        
        zillow_data.append(data)

In [83]:
def create_zillow_df(data_list: list) -> pd.DataFrame:
    """
    Formats json zillow data into a pandas DataFram
    """
    df = pd.DataFrame()
    for i in data_list:
        for item in i['cat1']['searchResults']['listResults']:
            df = df.append(item, ignore_index=True)
    return df

In [84]:
final_data = create_zillow_df(zillow_data)

In [85]:
final_data

,address,addressCity,addressState,addressStreet,addressZipcode,area,availabilityDate,badgeInfo,baths,beds,...,zpid,builderName,hasOpenHouse,openHouseDescription,openHouseEndDate,openHouseStartDate,streetViewMetadataURL,streetViewURL,lotAreaString,isPropertyResultCDP
0,"143 Walnut St NW, Atlanta, GA 30314",Atlanta,GA,143 Walnut St NW,30314,1405,None,None,2.0,4,...,35889872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"650 Simmons St NW, Atlanta, GA 30318",Atlanta,GA,650 Simmons St NW,30318,1483,None,None,3.0,3,...,35833599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1150 Hubbard St SW, Atlanta, GA 30310",Atlanta,GA,1150 Hubbard St SW,30310,None,None,None,0.0,None,...,35891070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"2839 The Fontainebleau SW, Atlanta, GA 30331",Atlanta,GA,2839 The Fontainebleau SW,30331,1792,None,None,2.0,3,...,35869024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"3235 Roswell Rd NE UNIT 408, Atlanta, GA 30305",Atlanta,GA,3235 Roswell Rd NE UNIT 408,30305,980,None,None,1.0,1,...,55018912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,"524 Ivy Pl NE, Atlanta, GA 30342",Atlanta,GA,524 Ivy Pl NE,30342,3856,None,None,5.0,5,...,65452324,NaN,1.0,Open House - 1:00 - 3:00 PM,2023-02-19T15:00:00,2023-02-19T13:00:00,NaN,NaN,NaN,NaN
236,"Cosmos Plan, Aviary Park",Atlanta,GA,"Cosmos Plan, Aviary Park",30311,1421,None,None,3.0,3,...,2061571205,Starlight,1.0,Open House - 8:00 - 8:30 AM,2023-02-18T08:30:00,2023-02-18T08:00:00,NaN,NaN,NaN,1.0
237,"1443 Mozley Pl SW, Atlanta, GA 30314",Atlanta,GA,1443 Mozley Pl SW,30314,2278,None,None,3.0,5,...,35843872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,"1951 Akron Dr SE, Atlanta, GA 30315",Atlanta,GA,1951 Akron Dr SE,30315,1025,None,None,2.0,3,...,35887354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
final_data.to_csv('sample-zillow.csv', index=False)